In [1]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [2]:
DATA_IN_PATH = './data/'
DATA_OUT_PATH = './output/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

input_data = np.load(open(DATA_IN_PATH + INPUT_TRAIN_DATA, 'rb'))
label_data = np.load(open(DATA_IN_PATH + LABEL_TRAIN_DATA, 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [3]:
TEST_SPLIT = 0.1
RNG_SEED = 13371447
VOCAB_SIZE = prepro_configs['vocab_size']+1
EMB_SIZE = 128
BATCH_SIZE = 16
NUM_EPOCHS = 1

input_train, input_eval, label_train, label_eval = train_test_split(input_data, label_data, test_size=TEST_SPLIT, random_state=RNG_SEED)

In [4]:
def mapping_fn(X, Y):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=len(input_train))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
    dataset = dataset.shuffle(buffer_size=len(input_eval))
    dataset = dataset.batch(16)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [5]:
def model_fn(features, labels, mode, params):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embedding_layer = tf.keras.layers.Embedding(
                    VOCAB_SIZE,
                    EMB_SIZE)(features['x'])

    dropout_emb = tf.keras.layers.Dropout(rate = 0.2)(embedding_layer)
    
    conv = tf.keras.layers.Conv1D(
           filters=32,
           kernel_size=3,
           padding='same',
           activation=tf.nn.relu)(dropout_emb)
  
    pool = tf.keras.layers.GlobalMaxPool1D()(conv)

    hidden = tf.keras.layers.Dense(units=250, activation=tf.nn.relu)(pool)   


    dropout_hidden = tf.keras.layers.Dropout(rate=0.2)(hidden, training = TRAIN)
    logits = tf.keras.layers.Dense(units=1)(dropout_hidden)

    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        
    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        )

In [6]:
est = tf.estimator.Estimator(model_fn, model_dir="output/checkpoint/korean_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'output/checkpoint/korean_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001BFE6F5F710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
time_start = datetime.utcnow()
print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 

est.train(train_input_fn)

time_end = datetime.utcnow()
print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))

Experiment started at 13:32:36
.......................................
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into output/checkpoint/korean_model\model.ckpt.
INFO:tensorflow:loss = 0.6906611, step = 1
INFO:tensorflow:global_step/sec: 17.0539
INFO:tensorflow:loss = 0.66531324, step = 101 (5.864 sec)
INFO:tensorflow:global_step/sec: 17.4508
INFO:tensorflow:loss = 0.40304634, step = 201 (5.730 sec)
INFO:tensorflow:global_step/sec: 18.3844
INFO:tensorflow:loss = 0.5088785, step = 301 (5.439 sec)
INFO:tensorflow:global_step/sec: 18.13
INFO:tensorflow:loss = 0.7183562, step = 401 (5.518 sec)
INFO:tensorflow:global_step/sec: 18.1139
INFO:tensorflow:loss = 0.2504524, step = 501 (5.519 sec)
INFO:tensorflow:global_step/sec: 18.4123
INFO:tensorflow:loss = 0.5646

INFO:tensorflow:loss = 0.33040154, step = 7901 (5.584 sec)
INFO:tensorflow:global_step/sec: 15.9528
INFO:tensorflow:loss = 0.4905103, step = 8001 (6.272 sec)
INFO:tensorflow:global_step/sec: 15.0087
INFO:tensorflow:loss = 0.36856169, step = 8101 (6.658 sec)
INFO:tensorflow:global_step/sec: 14.706
INFO:tensorflow:loss = 0.31347573, step = 8201 (6.800 sec)
INFO:tensorflow:global_step/sec: 17.3402
INFO:tensorflow:loss = 0.29515153, step = 8301 (5.771 sec)
INFO:tensorflow:global_step/sec: 17.422
INFO:tensorflow:loss = 0.29949865, step = 8401 (5.735 sec)
INFO:tensorflow:Saving checkpoints for 8438 into output/checkpoint/korean_model\model.ckpt.
INFO:tensorflow:Loss for final step: 0.166874.
.......................................
Experiment finished at 13:40:37

Experiment elapsed time: 480.793581 seconds


In [8]:
valid = est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-29-13:40:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/checkpoint/korean_model\model.ckpt-8438
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-29-13:40:39
INFO:tensorflow:Saving dict for global step 8438: acc = 0.8218667, global_step = 8438, loss = 0.40148613
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8438: output/checkpoint/korean_model\model.ckpt-8438


In [9]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'

test_input_data = np.load(open(DATA_IN_PATH + INPUT_TEST_DATA, 'rb'))
test_label_data = np.load(open(DATA_IN_PATH + LABEL_TEST_DATA, 'rb'))

In [10]:
def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((test_input_data, test_label_data))
    dataset = dataset.batch(16)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [11]:
predict = est.evaluate(test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-29-13:42:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/checkpoint/korean_model\model.ckpt-8438
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-29-13:42:47
INFO:tensorflow:Saving dict for global step 8438: acc = 0.81692, global_step = 8438, loss = 0.40289745
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8438: output/checkpoint/korean_model\model.ckpt-8438


In [12]:
predict

{'acc': 0.81692, 'loss': 0.40289745, 'global_step': 8438}